In [118]:
import copy
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# "batch norm analog" - i.e. collector neuron - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# TODO: 
#  3. Collect stats on gradients, neuron values, over training step
#  4. Fix scrolling
#  5. Use Cuda
#  6a. Relu -> Relu6
#  6b. Improve death, introduce train mode/ death mode (kill any which reach zero)
#  7. Add analog of batch norm (additive parameter in DestinationNeuron (need to make AdStep optimiser to support not-scalar values for that)
# After that can think on Cifar10 convnet death

import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data
import importlib
from ai_libs.basic.deep_reload import deep_reload
import livenet
deep_reload(livenet)
import ai_libs.simple_log as simple_log
from ai_libs.simple_log import LOG
simple_log.level = simple_log.LogLevel.INFO
import math
from matplotlib import pyplot as plt
plt.ion()
%matplotlib TkAgg
livenet.backend.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4


2.3.1+cu121


'%.4f'

In [119]:
downscale = (4, 4)
test = livenet.datasets.get_cifar10_test()
train = livenet.datasets.get_cifar10_train()
test_x, test_y = livenet.datasets.to_plain(*test, downscale=downscale, to_gray=True, to_odd=True)
train_x, train_y = livenet.datasets.to_plain(*train, downscale=downscale, to_gray=True, to_odd=True)


Files already downloaded and verified
Files already downloaded and verified


In [120]:
network = livenet.nets.create_perceptron(train_x.shape[1], 8, 2)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)

LiveNet


In [145]:
# simple_log.level = simple_log.LogLevel.DEBUG
network.mortal = True
network.context.regularization_l1 = 0.0005
optimizer.learning_rate = 0.002
trainer.step(1000)


Iˈ0.000 0.874 = 0.864+0.010reg params=80 tick 1201 livenet/net_trainer.py:94
Iˈ0.094 0.886 = 0.875+0.010reg params=80 tick 1202 livenet/net_trainer.py:94
Iˈ0.220 0.864 = 0.854+0.010reg params=80 tick 1203 livenet/net_trainer.py:94
Iˈ0.368 0.885 = 0.874+0.010reg params=80 tick 1204 livenet/net_trainer.py:94
Iˈ0.511 0.867 = 0.857+0.010reg params=80 tick 1205 livenet/net_trainer.py:94
Iˈ0.655 0.891 = 0.880+0.010reg params=80 tick 1206 livenet/net_trainer.py:94
Iˈ0.839 0.854 = 0.843+0.010reg params=80 tick 1207 livenet/net_trainer.py:94
Iˈ0.985 0.880 = 0.869+0.010reg params=80 tick 1208 livenet/net_trainer.py:94
Iˈ1.130 0.880 = 0.869+0.010reg params=80 tick 1209 livenet/net_trainer.py:94
Iˈ1.250 0.857 = 0.847+0.010reg params=80 tick 1210 livenet/net_trainer.py:94
Iˈ1.353 0.891 = 0.881+0.010reg params=80 tick 1211 livenet/net_trainer.py:94
Iˈ1.483 0.883 = 0.872+0.010reg params=80 tick 1212 livenet/net_trainer.py:94
Iˈ1.624 0.900 = 0.889+0.010reg params=80 tick 1213 livenet/net_trainer.py:94

KeyboardInterrupt: 

In [146]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y.numpy() - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff)}"


'accuracy 0.6769'

In [32]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["N0->D0"].item()
        # val0 = param["loss"]
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

#plt.figure(figsize=(16, 9))
plt.grid()
values = get_param_values(trainer.history, param_picker1)
plt.plot(values)
# accum = backend.stat_utils.AccumStat()
# accum.add_value(trainer.network.parameters())
# accum.plot()


Using matplotlib backend: tkagg


In [117]:
print(network.context.topology_stat.get_stat())
for name, p in sorted(network.named_parameters(),key=lambda x: x[0]):
    print(name, f"{p.item():.2f}")


{'dangle': {'RegularNeuron': 0, 'DestinationNeuron': 0}, 'useless': {'RegularNeuron': 0, 'InputNeuron': 57}}
D0 0.39
D1 -0.43
I0->N1 0.85
I0->N3 -0.99
I11->N3 0.59
I19->N1 -0.73
I41->N1 -0.29
I44->N3 0.15
I46->N1 -0.41
I63->N3 0.25
N1 -0.07
N1->D0 -1.15
N1->D1 0.94
N3 0.04
N3->D0 -1.49
N3->D1 0.81


In [10]:
def build_symmetric_dangle_net():
    from livenet.backend.core import DestinationNeuron, RegularNeuron, InputNeuron
    net = livenet.core.livenet.LiveNet()
    net.outputs += [DestinationNeuron(net.context, activation=None), DestinationNeuron(net.context, activation=None)]
    net.inputs += [InputNeuron(net.context)]
    neuron = RegularNeuron(net.context, activation=torch.nn.ReLU())
    neuron.connect_to(net.outputs[0])
    neuron.connect_to(net.outputs[1])
    # net.inputs[0].connect_to(neuron)
    net.root.visit_member("init_weight")
    with torch.no_grad():
        net.outputs[0].b[...] = -30.0
        net.outputs[1].b[...] = -0
        #net.outputs[1].b.requires_grad = False
        neuron.b[...] = 6.0
        neuron.axons[0].k[...] = 2.0
        neuron.axons[1].k[...] = -0
        #neuron.axons[1].k.requires_grad = False
    return net

train_x, train_y = livenet.datasets.get_odd_2()  
test_x, test_y = train_x, train_y
# train_x is not actually used



In [11]:
network = build_symmetric_dangle_net()
# train_x is not actually used as networks input doesn't connected to anything
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=3)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=1)


LiveNet


In [15]:
network.context.regularization_l1 = 0.05
optimizer.learning_rate = 0.002
trainer.step(15000)



Iˈ0.000 1.059 = 0.918+0.141reg params=5 livenet/net_trainer.py:93
Iˈ0.001 1.059 = 0.919+0.141reg params=5 livenet/net_trainer.py:93
Iˈ0.002 1.059 = 0.918+0.141reg params=5 livenet/net_trainer.py:93
Iˈ0.003 1.059 = 0.919+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.004 1.059 = 0.918+0.141reg params=5 livenet/net_trainer.py:93
Iˈ0.005 1.059 = 0.919+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.006 1.059 = 0.918+0.141reg params=5 livenet/net_trainer.py:93
Iˈ0.007 1.059 = 0.919+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.008 1.059 = 0.918+0.141reg params=5 livenet/net_trainer.py:93
Iˈ0.009 1.059 = 0.919+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.010 1.059 = 0.918+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.011 1.059 = 0.919+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.012 1.059 = 0.918+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.013 1.059 = 0.919+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.014 1.059 = 0.918+0.140reg params=5 livenet/net_trainer.py:93
Iˈ0.015 1.

In [143]:
life_stat = network.context.life_stat
df = pd.DataFrame(life_stat)
df2 = df[(df["type"] == "parameter") & (df["name"] == "I12->N7")]
#df2 = df[(df["type"] == "parameter") & (df["name"] == "D0")]

#df2 = df[(df["type"] == "delta") & (df["name"] == "N0->D0")]
#df2 = df[(df["type"] == "delta") & (df["name"] == "D0")]
#df2 = df[(df["type"] == "gradient") & (df["name"] == "N0->D0")]
#df2 = df[(df["type"] == "gradient") & (df["name"] == "D0")]
#df2 = df[(df["type"] == "max_output") & (df["name"] == "D0")]
# df2 = df[(df["type"] == "output_low_cut_ratio") & (df["tick"] == 199)]
# df2 = df[(df["type"] == "parameter") ]
df2 = df[(df["type"] == "output_low_cut_ratio")]
gps = df2.groupby("tick")
res = []
for name, group in gps:
    res.append({"tick": name, "value": group["value"].min()})
df2 = pd.DataFrame(res)
df2["value"].plot(grid=True, use_index=False, style='.-')
# df2 = df[(df["type"] == "output_high_cut_ratio")]
#df2 = df2.groupby("tick", as_index=False).max()

<Axes: >

In [131]:
df2 = df[df["type"] == "parameter"]
df2.loc[df2["value"].abs().idxmax()]

type                parameter
value               -1.385234
name                   N7->D0
class_name    AdStepParameter
tick                      997
Name: 488194, dtype: object

In [54]:
t = torch.tensor(0.0)

1